# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [118]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [133]:
# Get current folder and subfolder event data
filepath = os.getcwd()+'/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    # remove ipynb_checkpoints from string, to return the correct csv.
    file_path_list = glob.glob(os.path.join(root.replace(".ipynb_checkpoints",
                                                                    ""),'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [134]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        # skip headres
        next(csvreader)
        
        # extracting each data row one by one and append it
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv 
# that will be used to insert data into the 
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstname','gender','iteminSession','lastname',\
                'length','level','location','sessionid','song','userid'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], 
                                    row[7], row[8], row[12], row[13], row[16]))

In [135]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [136]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [137]:
# TO-DO: Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' :1}"""
    )
except Exception as e:
    print(e)


#### Set Keyspace

In [138]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

We will first create the table 'song_session' which consists of 5 columns:

1. sessionId (int): session id
2. itemInSession (int): item number in session 
3. artist (text):  artist name
4. song (text): song title
5. length (float): song's length

Partition key = sessionId
composite key = itemInSession

In [139]:
# CREATING THE TABLE

query = "CREATE TABLE IF NOT EXISTS song_session"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                    PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [140]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [141]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

# To answer Query-1,  
# Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, 
# and itemInSession = 4

query= "SELECT artist, song, length FROM song_session WHERE sessionId=338 and itemInSession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We will first create the table 'artist_session' which consists of 6 columns:

1. userId (int): user id
2. sessionId (int): session id
3. itemInSession (int): item number in session
4. artist (text): artist name
5. song (text): song title
6. user (text): first and last name of the user



We use 'userId' and 'sessionId' as the composite partition keys and 'itemInSession' as the clustering key to allow sorting by itemInSession

In [142]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and 
# user (first and last name) for userid = 10, sessionid = 182

query= "CREATE TABLE IF NOT EXISTS artist_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, user text, \
                PRIMARY KEY((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)     
                    

In [143]:
# INSERT DATA
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_session (userId, sessionId, itemInSession, artist, song, user)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

In [144]:
# Adding the SELECT statement to verify the data was entered into the table

# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
    

query = "SELECT itemInSession, artist, song, user FROM artist_session WHERE userId = 10 and sessionId = 182 ;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.user)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

We will create the table user_song which consists of 3 columns:

1. song (text): song title
2. userId (int): user id
3. user (text): first and last name of user

Partition key = song 
composite key is userId. It is also added as primary key to ensure the row data is unique.

In [145]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#CREATING THE TABLE

query= "CREATE TABLE IF NOT EXISTS user_song"
query = query + "(song text, userId int, user text, PRIMARY KEY(song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)     

In [146]:
# INSERTING DATA
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    # insert the lines in the file into the table music_song
    for line in csvreader:
        query = "INSERT INTO user_song(song, userId, user)"
        query = query + "VALUES(%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),  line[1] + " " + line[4]))

In [147]:
# VERIFY DATA WAS ENTERED INTO TABLE

#Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "select user from user_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [148]:
## TO-DO: Drop the table before closing out the sessions

query = "drop table song_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [149]:
session.shutdown()
cluster.shutdown()